In [ ]:
#default_exp s3_scrape

In [ ]:
#export

import boto3
import pandas as pd
import os
from PIL import Image
import json

In [ ]:
#export

def scrape_bucket(bucket_name):
    s3_resource = boto3.resource('s3')
    s3_client = boto3.client('s3')
    bucket = s3_resource.Bucket(bucket_name)
    
    volume_ids = []
    titles = []
    volume_roots = []
    image_counts = []
    has_jpg = []
    has_tif = []
    has_other = []
    other = []
    has_pdf = []
    has_metadata = []
    volume_metadata = []
    
    folders = ["jpg", "tif", "metadata"]    
    
    #for obj in bucket.objects.all():
    for obj in bucket.objects.filter(Prefix = "Brazil/Rio_de_Janeiro"):    
        if (len(obj.key.split('/')) >= 5) and (obj.key.split('/')[4] != '') and (obj.key.split('/')[4] not in volume_ids):            
            volume_ids.append(obj.key.split('/')[4])
            volume_root = obj.key[:obj.key.find('/', obj.key.find(obj.key.split('/')[4]))]
            volume_roots.append(volume_root)            
            has_metadata.append(False)
            has_pdf.append(False)
            has_jpg.append(False)
            has_tif.append(False)
            has_other.append(False)
            other.append(None)
            for volume_obj in bucket.objects.filter(Prefix = volume_root):
                if "DC.xml" in volume_obj.key:                    
                    has_metadata[-1] = True
                    s3_client.download_file(bucket.name, volume_obj.key, "temp.xml")
                    vol_dict = ssda_volume_xml_to_dict("temp.xml", volume_root)
                    volume_metadata.append(vol_dict)
                    if "title" in vol_dict:
                        titles.append(vol_dict["title"])
                    else:
                        titles.append("no title")
                    os.remove("temp.xml")
                elif "pdf" in volume_obj.key.lower():
                    has_pdf[-1] = True
                elif (has_jpg[-1] == False) and (volume_obj.key.lower().split('/')[5] == "jpg"):
                    has_jpg[-1] = True
                elif (has_tif[-1] == False) and (volume_obj.key.lower().split('/')[5] == "tif"):
                    has_tif[-1] = True
                elif (len(volume_obj.key.split('/')) > 6) and (volume_obj.key.lower().split('/')[5] not in folders) and ((other[-1] == None) or (volume_obj.key.lower().split('/')[5] not in other[-1])):
                    has_other[-1] = True
                    if other[-1] == None:
                        other[-1] = volume_obj.key.lower().split('/')[5]
                    else:
                        other[-1] = other[-1] + '|' + volume_obj.key.lower().split('/')[5]
                
            image_metadata = []
            prod_imgs = None
            if has_jpg[-1]:
                prod_imgs = "jpg"
            elif has_tif[-1]:
                prod_imgs = "tif"
                
            if prod_imgs == None:
                volume_metadata[-1]["images"] = []
                image_counts.append(0)
            else:                
                for image_obj in bucket.objects.filter(Prefix = volume_root + '/' + prod_imgs.upper()):
                    if ('.' + prod_imgs) in image_obj.key.lower():
                        file_name = image_obj.key[image_obj.key.rfind('/') + 1:image_obj.key.rfind('.')]
                        if not file_name.isdigit():
                            print("found bad image file name at " + image_obj.key)
                            continue
                        extension = image_obj.key[image_obj.key.rfind('.') + 1:]
                        temp_path = file_name + '.' + extension
                        s3_client.download_file(bucket.name, image_obj.key, temp_path)
                        im = Image.open(temp_path)
                        width, height = im.size
                        im.close()
                        os.remove(temp_path)
                        image = {"file_name": int(file_name), "extension": extension, "height": height, "width": width}
                        image_metadata.append(image)
                volume_metadata[-1]["images"] = image_metadata
                image_counts.append(len(image_metadata))           
            
            print("Completed " + titles[-1])
                
    volumes_dict = {"id": volume_ids, "title": titles, "images": image_counts, "s3 root": volume_roots, "metadata": has_metadata, "has pdf": has_pdf, "has jpg": has_jpg, "has tif": has_tif, "has other": has_other, "other": other}
    volumes_df = pd.DataFrame.from_dict(volumes_dict)
    
    return volumes_df, volume_metadata

In [ ]:
#export

def ssda_volume_xml_to_dict(volume_xml, s3_path):
    import xml.etree.ElementTree as ET
    tree = ET.parse(volume_xml)
    root = tree.getroot()
    volume_dict = {}
    volume_dict["s3_path"] = s3_path
    for item in root:
        if item.text == None:
            volume_dict[item.tag] = None
            continue
        if item.text[0] == ' ':
            item.text = item.text[1:]        
        if item.tag == "subject":
            if "subject" in volume_dict:
                volume_dict["subject"].append(item.text.split("--"))
            else:
                volume_dict["subject"] = item.text.split("--")
        elif item.tag == "title":
            volume_dict["title"] = item.text
        elif item.tag == "contributor":
            if (item.text.find('(') != -1) and (item.text.find(')') != -1):
                name = item.text[:item.text.find('(')]
                role = item.text[item.text.find('(') + 1:item.text.find(')')]
            else:
                continue
            if "contributor" in volume_dict:
                volume_dict["contributor"].append({"name": name, "role": role})
            else:
                volume_dict["contributor"] = [{"name": name, "role": role}]
        elif item.tag == "identifier":
            volume_dict["identifier"] = item.text[item.text.find(':') + 1:]
        elif item.tag == "coverage":
            if ('.' in item.text) and (',' in item.text):
                if "coverage" in volume_dict:
                    volume_dict["coverage"]["coords"] = item.text
                else:
                    volume_dict["coverage"] = {"coords": item.text}
            elif "--" in item.text:
                places = item.text.split("--")
                if len(places) == 4:
                    if "coverage" in volume_dict:
                        volume_dict["coverage"]["country"] = places[1]
                        volume_dict["coverage"]["state"] = places[2]
                        volume_dict["coverage"]["city"] = places[3]
                    else:
                        volume_dict["coverage"] = {"country": places[1], "state": places[2], "city": places[3]}                
        elif item.tag == "source":
            if "coverage" in volume_dict:
                volume_dict["coverage"]["institution"] = item.text
            else:
                volume_dict["coverage"] = {"institution": item.text}
        elif ((item.tag == "type") and (item.text == "Text")) or (item.tag == "rights"):
            continue
        elif (item.tag == "creator") and (';' in item.text):
            creators = item.text.split(';')
            for creator in creators:
                if "creator" in volume_dict:
                    volume_dict["creator"].append(creator)
                else:
                    volume_dict["creator"] = [creator]
        else:
            if item.tag in volume_dict:                
                volume_dict[item.tag].append(item.text)                
            else:
                volume_dict[item.tag] = [item.text]       
        
        if "coverage" not in volume_dict:
            volume_dict["coverage"] = {}
            if "country" not in volume_dict["coverage"]:            
                volume_dict["coverage"]["country"] = volume_dict["s3_path"].split('/')[0].replace('_', ' ')
                volume_dict["coverage"]["state"] = volume_dict["s3_path"].split('/')[1].replace('_', ' ')
                volume_dict["coverage"]["city"] = volume_dict["s3_path"].split('/')[2].replace('_', ' ')
        
    return volume_dict

In [ ]:
#no_test

nova_lima_df, nova_lima_metadata = scrape_bucket("ssda-assets")

Completed Supplemento do Livro 8 para os assentos de óbitos dos escravos da Companhia do Morro Velho
Completed Livro de bautismos, 1846-1869
Completed Livro 6 de Óbitos, 1853-1859
Completed Livro de Óbitos, 1877-1892
Completed Livro 5 de Bautismos, 1831-1834
Completed Livro de bautismos, 1886-1890
Completed Livro de casamentos, 1843-1851
Completed Livro 8 de casamentos, 1891-1905
Completed Livro 6 de Casamentos, 1851-1891


In [ ]:
#no_test

nova_lima_df.to_csv("nova_lima.csv", index = False)
nova_lima_df.head()

,id,title,images,s3 root,metadata,has pdf,has jpg,has tif,has other,other
0,2223,Supplemento do Livro 8 para os assentos de óbi...,69,Brazil/Minas_Gerais/Nova_Lima/Nossa_Senhora_do...,True,True,True,False,False,[]
1,39001,"Livro de bautismos, 1846-1869",400,Brazil/Minas_Gerais/Nova_Lima/Nossa_Senhora_do...,True,True,True,False,False,[]
2,39003,"Livro 6 de Óbitos, 1853-1859",194,Brazil/Minas_Gerais/Nova_Lima/Nossa_Senhora_do...,True,True,True,False,False,[]
3,39005,"Livro de Óbitos, 1877-1892",135,Brazil/Minas_Gerais/Nova_Lima/Nossa_Senhora_do...,True,True,True,False,False,[]
4,39008,"Livro 5 de Bautismos, 1831-1834",192,Brazil/Minas_Gerais/Nova_Lima/Nossa_Senhora_do...,True,True,True,False,False,[]


In [ ]:
#no_test

with open("nova_lima.json", "w") as outfile:
    outfile.write('{\n\"volumes\": \n')
    json.dump(nova_lima_metadata, outfile)
    outfile.write('}')

In [ ]:
#no_test

paraiba_df, paraiba_metadata = scrape_bucket("ssda-assets")

Completed Land Grants (sesmarias), 1728 -1738
Completed Plots of Land - 1714-1717 / Land grants (sesmarias) - 1714-1717
Completed Land Grants (sesmarias), 1742 - 1746
Completed Land Grant - Various Parishes, 1768 - 1776
Completed Land Grants (sesmarias), 1747 - 1755
Completed Plots of Land 1722-1727 / Land Grants (sesmarias) 1722-1727
Completed Land Grants (sesmarias), 1789 - 1808
Completed Land Titles from the Parish of Nossa Senhora dos Remédios da Vila de Sousa, 1854 - 1856
Completed Land Grants (sesmarias), 1816 - 1824
Completed Land Titles from the Parish of Sousa, 1859
Completed Land Titles from the Parish of Nossa Senhora da Guia da Vila de Patos, 1855 - 1856
Completed Land Titles from the Parish of  Nossa Senhora das Neves, 1854 - 1861
Completed Land Titles from the Parish of Cabaceiras, 1855 - 1862
Completed Land Titles from the Parish of  Independência, 1856 -1865
Completed Land Titles from the Parish of Cidade de Sousa, 1856
Completed Land Titles from Cuité, 1856
Completed L

Completed Chief of Police: Letters to Deputies ("delegados") and Assistant Deputies ("subdelegados"),  1861-1862
Completed Correspondence of the Public Education Board of Directors, 1864-1865
Completed Register of Public School Teachers, 1852-1865
Completed Secretary of Police: Correspondence with Provincial Presidency, 1870-1871 
Completed Public Education: correspondence, 1886
Completed Muncipal Council (Câmara) Treasury Records, 1847-1853
Completed Ministry of Justice: Payroll, 1837-1838
Completed General Council of the Province of Paraíba do Norte: projects of the Provincial Legislative Assembly, 1829-1830
Completed Correspondence between the Treasury Inspector and Provincial Presidents, 1850-1853
Completed Correspondence of the Provincial Treasury with the Consulate, Tax Agencies, Fiscal Prosecution and Inspection, 1857-1865
Completed Provincial Treasury: balance sheet, 1857 (Third Section of the Provincial Public Treasury)
Completed Provincial Treasury: register of expenses, 1858

Completed Book of Baptisms [of the] Freguesia de Nossa Senhora dos Milagres, Cariri (1765-1771) / Livro de Batismos [da] Freguesia de Nossa Senhora dos Milagres, Cariri de Fora 1765-1771
Completed Book 3: Baptisms (1773-1787) / Livro n. 3 Batizados de 1773-1787
Completed Book 4 of the Freguesia de Nossa Senhora dos Milagres do Cariri de Fora (1787-1793) / Freguesia de Nossa Senhora dos Milagres do Cariri de Fora Livro n. 4 1787 a 1793
Completed Book 8 of the Freguesia de Nossa Senhora dos Milagres do Cariri de Fora (1825-1837) / Freguesia de Nossa Senhora dos Milagres do Cariri de Fora Livro n. 8 1825 a 1837
Completed Book 7: Baptisms (1835-1846) / Livro nº 7 Batizados 1835 - 1846
Completed Book 10: Baptisms (1858-1862) / Livro 10 De 1858 a 1862 Batismo
Completed Book 8: Baptisms (1846-1853) / Livro N. 8 1846-1853 Batizados
Completed Book 9: 1853-1858 / Livro 9 1853 - 1858
Completed Book 12 (formerly book 19) (1864-1867) / Livro 12 (Ex 19) De 1864 - 1867
Completed Book 14 (formerly boo

In [ ]:
#no_test

paraiba_df.to_csv("paraiba.csv", index = False)
paraiba_df.head()

,id,title,images,s3 root,metadata,has pdf,has jpg,has tif,has other,other
0,315081,"Land Grants (sesmarias), 1728 -1738",410,Brazil/Paraíba/João_Pessoa/Arquivo_Histórico_d...,True,False,True,True,True,"[cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, ..."
1,315492,Plots of Land - 1714-1717 / Land grants (sesma...,406,Brazil/Paraíba/João_Pessoa/Arquivo_Histórico_d...,True,False,True,True,True,"[cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, ..."
2,316562,"Land Grants (sesmarias), 1742 - 1746",462,Brazil/Paraíba/João_Pessoa/Arquivo_Histórico_d...,True,False,True,True,True,"[cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, ..."
3,317025,"Land Grant - Various Parishes, 1768 - 1776",464,Brazil/Paraíba/João_Pessoa/Arquivo_Histórico_d...,True,False,True,True,True,"[cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, ..."
4,317490,"Land Grants (sesmarias), 1747 - 1755",386,Brazil/Paraíba/João_Pessoa/Arquivo_Histórico_d...,True,False,True,True,True,"[cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, cr2, ..."


In [ ]:
#no_test

with open("paraiba.json", "w") as outfile:
    outfile.write('{\n\"volumes\": \n')
    json.dump(paraiba_metadata, outfile)
    outfile.write('}')

In [ ]:
#no_test

rio_df, rio_metadata = scrape_bucket("ssda-assets")

ClientError: An error occurred (ExpiredToken) when calling the ListObjects operation: The provided token has expired.

In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()